# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
pip install -r requirements.txt --trusted-host mirrors.aliyun.com -i http://mirrors.aliyun.com/pypi/simple/

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 21.8 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 21.0 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 18.6 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 24.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.3 MB/s eta 0:0

In [2]:
import os
# 设置环境变量，下载器就会使用镜像站
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [3]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [4]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.545 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [5]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [9]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [10]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [11]:
print(response[0].split("### Response:")[1])


<think>
好，我现在需要分析这位28岁的男性，是一名程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心的情况。首先，我会考虑他的可能健康问题。头晕和脖子疼，有时候还恶心，这可能与脑力活动有关，因为程序员需要长时间思考和工作。常见的原因包括中风、脑损伤、脑血管疾病或者脑力活动过载导致的疲劳。

接下来，我会思考可能的健康问题。首先，中风是可能的，因为长时间的脑力活动可能导致脑膜炎，进而导致中风。其次，脑损伤，比如脑膜炎、外伤、脑膜炎等，也可能导致头晕和脖子疼。此外，脑血管疾病，比如脑中风、血脑屏障受损等，也可能引起类似的症状。还有可能是脑力过度疲劳，导致疲劳症状。还有一种可能是精神压力，虽然通常不是直接导致头晕和脖子疼的原因，但可能在某些情况下出现。

然后，我会考虑可能的诊断。首先，考虑中风。中风通常表现为头晕、恶心、呕吐、呼吸困难、意识模糊等，这些症状与患者的情况相吻合。其次，脑损伤可能需要进一步的脑部影像学检查，如MRI或CT scan，以确认具体的损伤类型。此外，脑血管疾病可能需要进行血管检查，如血钙试验、血常规等，以评估血管的健康状况。精神压力可能需要进行心理评估，以确定是否存在焦虑或抑郁的倾向。

然后，我会考虑可能的治疗方案。首先，中风的治疗通常包括药物治疗，如抗恶心药物、抗抑郁药，以及神经药物如抗休制等。此外，可能需要药物治疗以缓解脑损伤，如抗恶心药物、抗抑郁药。在脑损伤的情况下，可能需要进一步的影像学检查，如CT scan或MRI，以明确损伤类型。对于血管疾病，可能需要进行血管检查，如血钙试验、血常规等，以评估血管的健康状况。精神压力可能需要进行心理评估，以确定是否存在焦虑或抑郁的倾向，然后根据情况采取适当的治疗措施。

最后，我会总结，这位患者的情况可能涉及中风、脑损伤或脑血管疾病，可能需要进一步的检查和治疗。建议他首先进行脑部的影像学检查，如CT scan或MRI，以明确具体的损伤类型。如果检查结果表明中风，那么药物治疗是必要的。如果检查结果表明脑损伤，那么可能需要进一步的影像学检查和药物治疗。对于血管疾病，可能需要进行血管检查，评估血管的健康状况。如果精神压力较大，可能需要进行心理评估，然后采取适当的治疗措施。
</think>

男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。可能的健康问题包括

---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [12]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [13]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft_Chinese.json: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [14]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [16]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [18]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth: Already have LoRA adapters! We shall skip this step.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [19]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [20]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.161300
2,3.169600
3,3.079300
4,3.166400
5,3.150400
6,3.008400
7,2.984700
8,2.946100
9,2.900900
10,2.794600


TrainOutput(global_step=60, training_loss=2.4794784744580585, metrics={'train_runtime': 703.3323, 'train_samples_per_second': 10.919, 'train_steps_per_second': 0.085, 'total_flos': 7.163728311484416e+16, 'train_loss': 2.4794784744580585})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [21]:
model.save_pretrained("qwen-1.5b_lora_model")

In [22]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [23]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [25]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人，5天已经得了急性阑尾炎，还有一点腹痛和发热，但还是有压痛包块。哎，这个情况还挺麻烦的。我得好好想想怎么处理。

首先，压痛包块通常就是阑尾炎的一种表现，所以要先排除其他可能。但这个病人没有明显的感染史，也没有其他症状，只能先看这个包块是不是阑尾炎的表现。

然后，考虑到急性阑尾炎，应该立即处理。因为急性阑尾炎的诊断需要快速确定，不能拖延时间。所以，赶紧做影像学检查，比如超声或CT，看看是不是有感染，或者包块是否是阑尾炎。

如果包块看起来像是阑尾炎的，那得先排除感染。但是病人没有感染史，可能可以先做个B超检查。然后，如果发现包块有感染，就得进一步检查。

不过，先做个B超检查，还是一个好主意。因为B超能明确看到包块的位置，然后根据包块的大小和位置来确定是否是阑尾炎。

总之，要尽快进行检查，因为急性阑尾炎的诊断需要及时。不能拖延时间，所以要立刻处理。
</think>
对于一个患有急性阑尾炎的病人，已经发病5天，腹痛稍有减轻但仍然发热，并且在体检时发现右下腹有压痛的包块，应当立即采取措施进行检查。

首先，应进行B超检查以明确包块的位置和大小。根据包块的检查结果，可以初步判断是否为急性阑尾炎。如果包块的大小和位置符合阑尾炎的特征，应立即进行相应的诊断。

此外，考虑到急性阑尾炎的诊断需要快速确定，尽快进行影像学检查是必要的。如果包块有感染的迹象，应进一步进行进一步的检查以排除感染的可能性。

因此，建议尽快进行B超检查以明确包块的位置和大小，并根据检查结果进行相应的诊断。尽快的处理可以确保诊断的准确性，从而更好地指导治疗。


In [26]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [27]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个60岁男性患者出现右侧胸疼，X线检查显示右肋膈角消失，诊断为肺结核伴右侧胸腔积液，听起来挺严重。肺结核通常会导致肺部肿大和积液，所以这可能是个很严重的肺部问题。

我得想想，胸腔积液的性质是什么呢？嗯，通常情况下，胸腔积液可能会形成气胸，也就是胸腔内积聚的气体，这会导致气体胸痛。不过，肺结核伴胸腔积液，可能还会有其他症状，比如胸痛或者胸痛加重。

现在，我要找出一个实验室检查来了解胸腔积液的性质。嗯，胸腔积液的性质通常由气胸和气胸痛来判断。气胸是胸腔内气体膨胀导致的胸痛，而气胸痛则是胸腔内气体膨胀导致的胸痛加重。

所以，我觉得气胸和气胸痛可能更直接地反映出胸腔积液的性质。因为它们直接涉及到气胸的形成，也就是胸腔内气体膨胀导致的胸痛。

嗯，再想想，像肺结核这种肺部感染，通常会导致肺部肿大和肺部积液。但肺结核通常不是像肺结核那样直接导致气胸，而是肺部肿大和积液。哦，不，肺结核可能引起气胸。

等等，我好像有点混乱了。让我再理清楚。

肺结核的典型症状包括胸痛和胸痛加重，尤其是胸痛加重。在X线检查中，肺结核可能也会导致气胸，也就是胸腔内气体膨胀。

嗯，对，肺结核通常会导致气胸，所以气胸和气胸痛可能就是用来判断肺结核伴胸腔积液的性质的。

嗯，所以，我觉得气胸和气胸痛能更好地反映出胸腔积液的性质，因为它们直接关联到气胸的形成，也就是胸腔内气体膨胀导致的胸痛。

嗯，对啊，所以答案应该是气胸和气胸痛。
</think>
对于60岁男性患者出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液的情况，实验室检查中可以用于了解胸腔积液性质的可能是气胸和气胸痛。因为胸腔积液导致的气胸即胸腔内气体膨胀，是直接反映胸腔积液性质的指标。因此，气胸和气胸痛能够帮助判断肺结核伴胸腔积液的性质。


In [29]:
my_question = "对于一名 46 岁的男性患者，工作是程序员，常年熬夜，经常吃海鲜等食品，最近突然脚疼，感觉像骨折一样疼痛，修养一段时间有所好转，但是过一段时间又会疼痛，从外观看骨头并无异常，可能是什么问题？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这位46岁的男性患者，工作是程序员，长期熬夜，经常吃海鲜，这些食物对他的身体很不利，因为海鲜中的某些成分可能对心脏不好。而且，最近脚疼，感觉有点像骨折，这种疼痛感觉很严重，尤其是吃海鲜后，脚疼得更严重。可是，修养一段时间后，感觉好些，然后又又疼痛，这说明他的身体恢复起来可能有问题。

他的外观看骨好像没发现问题，说明骨头没有异常，但如果是这样，这可能就是骨头有问题了。但是，他吃海鲜后，脚疼更严重，这可能意味着他的身体在处理海鲜的过程中，导致了骨头的损伤，尤其是脚，这可能就是骨质增生的问题。可能是因为海鲜摄入过多，导致骨质增生，进而导致脚痛。

这种情况下，他的身体可能在处理海鲜的过程中，导致了骨头的问题。所以，他可能需要更专业的医疗咨询，或者在医生那里进一步检查，看看是不是因为海鲜摄入过多导致的骨质增生问题。

综上所述，他的脚疼是由于海鲜摄入过多导致骨质增生，这可能就是他的问题所在。
</think>
这位46岁的男性患者，工作是程序员，长期熬夜，吃海鲜，这些食物可能导致他的身体负担加重。最近脚疼，感觉像骨折一样，这可能是因为海鲜摄入过多导致的骨质增生。这种增生可能影响到脚，导致疼痛。外观看骨没有问题，但因为海鲜摄入过多，导致骨质增生，这可能就是骨质增生问题。这种情况需要医生更仔细地评估，并建议进一步检查，以确定具体原因。建议与医生详细讨论，以确保问题得到正确诊断和治疗。
